In [1]:
"""
Based on original code by Tony Lin : https://github.com/tnlin/PokemonGo-TSP    
Developed by Andre Pradika E. P - DC 101 LAB @ 18Feb19 
"""
import pandas as pd
import math
import numpy as np
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt

In [2]:
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a - b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [3]:
def sum_distmat(p, distmat):
    dist = 0
    num_location = p.shape[0]
    for i in range(num_location - 1):
        dist += distmat[p[i]][p[i + 1]]
    dist += distmat[p[0]][p[num_location - 1]]
    return dist

In [4]:
def get_distmat(p):
    num_location = p.shape[0]
    # 1 degree of lat/lon ~ 111km = 111000m
    p *= 111000
    distmat = np.zeros((num_location, num_location))
    for i in range(num_location):
        for j in range(i, num_location):
            distmat[i][j] = distmat[j][i] = np.linalg.norm(p[i] - p[j])
    return distmat

In [5]:
def swap(sol_new):
    while True:
        n1 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        n2 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        if n1 != n2:
            break
            
    sol_new[n1], sol_new[n2] = sol_new[n2], sol_new[n1]
    return sol_new

In [6]:
def reverse(sol_new):
    while True:
        n1 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        n2 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        if n1 != n2:
            break
    
    sol_new[n1:n2] = sol_new[n1:n2][::-1]
    return sol_new

In [7]:
def transpose(sol_new):
    while True:
        n1 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        n2 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        n3 = np.int(np.floor(np.random.uniform(0, sol_new.shape[0])))
        if n1 != n2 != n3 != n1:
            break
    # Let n1 < n2 < n3
    n1, n2, n3 = sorted([n1, n2, n3])

    # Insert data between [n1,n2) after n3
    tmplist = sol_new[n1:n2].copy()
    sol_new[n1: n1 + n3 - n2 + 1] = sol_new[n2: n3 + 1].copy()
    sol_new[n3 - n2 + 1 + n1: n3 + 1] = tmplist.copy()
    return sol_new

In [8]:
def accept(cost_new, cost_current, T):
    # If new cost better than current, accept it
    # If new cost not better than current, accept it by probability P(dE)
    # P(dE) = exp(dE/(kT)), defined by Metropolis
    return (cost_new < cost_current or 
            np.random.rand() < np.exp(-(cost_new - cost_current) / T))

In [9]:
def plot(path, points, costs):
    '''
    path: List of the different orders in which the nodes are visited
    points: coordinates for the different nodes
    costs: Cost of each iteration
    '''
    # Change figure size
    plt.figure(figsize=(15, 6))

    '''
    Plot Cost Function
    '''
    plt.subplot(121)
    curve, = plt.plot(np.array(costs), label='Distance(m)')
    plt.ylabel("Distance")
    plt.xlabel("Iteration")
    plt.grid(True)
    plt.legend()
    cost = str("%.2f" % round(costs[-1], 2))
    plt.title("Final Distance: " + cost)

    '''
    Plot TSP Route
    '''
    plt.subplot(122)
    # Transform back to longitude/latitude
    points = (points / 111000).tolist()

    # Unpack the primary path and transform it into a list of ordered coordinates
    x = [];
    y = []
    for i in path:
        x.append(points[i][1])
        y.append(points[i][0])
    x.append(points[path[0]][1])
    y.append(points[path[0]][0])

    # Plot line
    plt.plot(x, y, 'c-', label='Route')

    # Plot dot
    plt.plot(x, y, 'bo', label='Location')

    # Avoid scientific notation
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

    # Set axis too slightly larger than the set of x and y
    plt.xlim(min(x) * 0.99999, max(x) * 1.00001)
    plt.ylim(min(y) * 0.99999, max(y) * 1.00001)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("TSP Route Visualization")
    plt.grid(True)
    # plt.show()
    plt.savefig("1.png")

In [10]:
def plot(path, points, costs, counter):
    '''
    path: List of the different orders in which the nodes are visited
    points: coordinates for the different nodes
    costs: Cost of each iteration
    '''
    # Change figure size
    plt.figure(figsize=(15, 6))

    '''
    Plot Cost Function
    '''
    plt.subplot(121)
    curve, = plt.plot(np.array(costs), label='Distance(m)')
    plt.ylabel("Distance")
    plt.xlabel("Iteration")
    plt.grid(True)
    plt.legend()
    cost = str("%.2f" % round(costs[-1], 2))
    plt.title("Final Distance: " + cost)

    '''
    Plot TSP Route
    '''
    plt.subplot(122)
    # Transform back to longitude/latitude
    points = (points / 111000).tolist()

    # Unpack the primary path and transform it into a list of ordered coordinates
    x = [];
    y = []
    for i in path:
        x.append(points[i][1])
        y.append(points[i][0])
    x.append(points[path[0]][1])
    y.append(points[path[0]][0])

    # Plot line
    plt.plot(x, y, 'c-', label='Route')

    # Plot dot
    plt.plot(x, y, 'bo', label='Location')

    # Avoid scientific notation
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

    # Set axis too slightly larger than the set of x and y
    plt.xlim(min(x) * 0.99999, max(x) * 1.00001)
    plt.ylim(min(y) * 0.99999, max(y) * 1.00001)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title("TSP Route Visualization")
    plt.grid(True)
    # plt.show()
    plt.savefig(str(counter)+'.jpg')

In [ ]:

def main():
    # Use the corresponding data file
    filename = "data/data1.csv"
    coordinates = np.loadtxt(filename, delimiter=',')

    # Constant Definitions
    NUM_NEW_SOLUTION_METHODS = 3
    SWAP, REVERSE, TRANSPOSE = 0, 1, 2

    # Params Initial
    num_location = coordinates.shape[0]
    markov_step = 10 * num_location
    T_0, T, T_MIN = 100, 100, 1
    T_NUM_CYCLE = 1

    # Build distance matrix to accelerate cost computing
    distmat = get_distmat(coordinates)

    # States: New, Current and Best
    sol_new, sol_current, sol_best = (np.arange(num_location),) * 3
    cost_new, cost_current, cost_best = (float('inf'),) * 3

    # Record costs during the process
    costs = []

    # previous cost_best
    prev_cost_best = cost_best

    # counter for detecting how stable the cost_best currently is
    cost_best_counter = 0

    # Simulated Annealing
    while T > T_MIN and cost_best_counter < 150:
        for i in np.arange(markov_step):
            # Use three different methods to generate new solution
            # Swap, Reverse, and Transpose
            choice = np.random.randint(NUM_NEW_SOLUTION_METHODS)
            if choice == SWAP:
                sol_new = swap(sol_new)
            elif choice == REVERSE:
                sol_new = reverse(sol_new)
            elif choice == TRANSPOSE:
                sol_new = transpose(sol_new)
            else:
                print("ERROR: new solution method %d is not defined" % choice)

            # Get the total distance of new route
            cost_new = sum_distmat(sol_new, distmat)

            if accept(cost_new, cost_current, T):
                # Update sol_current
                sol_current = sol_new.copy()
                cost_current = cost_new

                if cost_new < cost_best:
                    sol_best = sol_new.copy()
                    cost_best = cost_new
            else:
                sol_new = sol_current.copy()

        # Lower the temperature
        alpha = 1 + math.log(1 + T_NUM_CYCLE)
        T = T_0 / alpha
        costs.append(cost_best)

        # Increment T_NUM_CYCLE
        T_NUM_CYCLE += 1

        # Detect stability of cost_best
        if isclose(cost_best, prev_cost_best, abs_tol=1e-12):
            cost_best_counter += 1
        else:
            # Not stable yet, reset
            cost_best_counter = 0

        # Update prev_cost_best
        prev_cost_best = cost_best

        # Monitor the temperature & cost
        print("Temperature:", "%.2f°C" % round(T, 2),
              " Distance:", "%.2fm" % round(cost_best, 2),
              " Optimization Threshold:", "%d" % cost_best_counter)

    # Show final cost & route
    print("Final Distance:", round(costs[-1], 2))
    print("Best Route", sol_best)

    # Plot cost function and TSP-route
    plot(sol_best.tolist(), coordinates, costs)


if __name__ == "__main__":
    main()